In [1]:
!pip install hopsworks==3.7.0 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 715.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
# !pip install catboost==1.0.2 --quiet
!pip install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.9 MB/s eta 0:00:00


In [3]:
import hopsworks
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
##load feature group from hopsworks
project = hopsworks.login()
fs = project.get_feature_store()


Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/112801
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
backfill_fg  = fs.get_feature_group(name="diabetes_features_backfill",
    version=1)

In [6]:
query = backfill_fg.select_all()
# query = fg.select(["age", "bmi", "HbA1c_level", "blood_glucose_level", "diabetes"])


In [7]:
feature_view = fs.get_or_create_feature_view(
    name="diabetes_training_feature_view",
    version=1,
    description="Feature view for diabetes training models",
    labels=["diabetes"],
    query=query
)

In [8]:
print(feature_view.query.to_string())

SELECT `fg0`.`patient_id` `patient_id`, `fg0`.`age` `age`, `fg0`.`hypertension` `hypertension`, `fg0`.`heart_disease` `heart_disease`, `fg0`.`bmi` `bmi`, `fg0`.`hba1c_level` `hba1c_level`, `fg0`.`blood_glucose_level` `blood_glucose_level`, `fg0`.`diabetes` `diabetes`, `fg0`.`datetime` `datetime`
FROM `churn_prediction_featurestore`.`diabetes_features_backfill_1` `fg0`


In [10]:
feature_df = feature_view.get_batch_data()
# feature_df.head(5)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.19s) 


,patient_id,age,hypertension,heart_disease,bmi,hba1c_level,blood_glucose_level,datetime
0,46496,45,0,0,46.27,6.5,299,2024-07-18 12:24:25.622299+00:00
1,126601,52,1,0,31.70,5.6,158,2024-07-18 12:24:25.622299+00:00
2,86567,24,0,0,20.97,5.8,145,2024-07-18 12:24:25.622299+00:00
3,132019,42,0,0,37.58,6.2,158,2024-07-18 12:24:25.622299+00:00
4,136388,75,1,1,32.14,7.0,237,2024-07-18 12:24:25.622299+00:00


In [31]:
td_version, td_job = feature_view.create_train_test_split(
    description = 'diabetes batch training dataset',
    data_format = "csv",
    test_size = 0.2,
    coalesce = True,
    statistics_config={
        "enabled": True,
        "histograms": False,
        "correlations": False
    }
)

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/112801/jobs/named/diabetes_training_feature_view_1_create_fv_td_18072024182126/executions


In [11]:
# td_version = 1
X_train, X_eval, y_train, y_eval = feature_view.get_train_test_split(training_dataset_version=td_version)

In [12]:
#  X_train.head(3)

,patient_id,age,hypertension,heart_disease,bmi,hba1c_level,blood_glucose_level,datetime
0,0,72,0,0,23.77,4.0,85,2024-07-18T15:49:55.081Z
1,1,79,1,0,41.20,6.5,144,2024-07-18T15:49:55.081Z
2,2,54,0,0,35.55,6.7,143,2024-07-18T15:49:55.081Z


In [13]:
# y_train.head(3)

,diabetes
0,0
1,1
2,1


In [14]:
#removing the patient_id and datetime
X_train = X_train.iloc[:,1:-1]
X_eval = X_eval.iloc[:,1:-1]

In [15]:
from catboost import CatBoostClassifier, Pool

In [16]:
train_pool = Pool(X_train, y_train, cat_features=None)
eval_pool = Pool(X_eval, y_eval, cat_features=None)

In [38]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    early_stopping_rounds=50
)

In [39]:
model.fit(train_pool, eval_set=eval_pool, verbose=100, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.9732624	best: 0.9732624 (0)	total: 15.5ms	remaining: 15.5s
100:	test: 0.9944194	best: 0.9944194 (100)	total: 1.23s	remaining: 11s
200:	test: 0.9946418	best: 0.9946447 (197)	total: 2.42s	remaining: 9.62s
300:	test: 0.9946546	best: 0.9946695 (275)	total: 3.61s	remaining: 8.39s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9946695347
bestIteration = 275

Shrink model to first 276 iterations.


In [40]:
y_pred = model.predict(X_eval)

In [41]:
report_dict = classification_report(y_eval, y_pred, output_dict=True)

In [42]:
print(classification_report(y_eval, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96     15611
           1       0.97      0.95      0.96     14956

    accuracy                           0.96     30567
   macro avg       0.96      0.96      0.96     30567
weighted avg       0.96      0.96      0.96     30567



<h3> Searching for better models </h3>

In [43]:
# param_grid = {
#     'iterations': [500, 1000, 1500],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'depth': [4, 6, 8, 10],
#     'l2_leaf_reg': [1, 3, 5, 7, 9],
#     'bagging_temperature': [0.1, 0.5, 1, 2, 5],
#     'random_strength': [1, 2, 5, 10],
#     'border_count': [32, 50, 100],
# }

# random_search = RandomizedSearchCV(
#     estimator=model,
#     param_distributions=param_grid,
#     n_iter=50,
#     scoring='f1',
#     cv=3,
#     verbose=3,
#     random_state=42,
#     n_jobs=-1
# )

#random_search.fit(X_train, y_train, cat_features=None)
# best_model = random_search.best_estimator_

# # Make predictions
# y_pred = best_model.predict(X_eval)
# y_proba = best_model.predict_proba(X_eval)[:, 1]

# # Evaluate the model
# print(classification_report(y_eval, y_pred))


# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(random_state=42)
# rf.fit(X_train, y_train.values.ravel())

# y_pred = rf.predict(X_eval)

# # Compare predictions (y_pred) with the labels in the test set (y_test)
# metrics = classification_report(y_eval, y_pred, output_dict=True)
# results = confusion_matrix(y_eval, y_pred)

In [44]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_eval, y_pred)
precision = precision_score(y_eval, y_pred)
recall = recall_score(y_eval, y_pred)
f1 = f1_score(y_eval, y_pred)


metrics = {
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "f1_score": f1
}

In [45]:
metrics

{'accuracy': 0.9624431576536788,
 'precision': 0.9725530458590007,
 'recall': 0.9500534902380315,
 'f1_score': 0.9611716160454576}

In [46]:
def register_python_model(model, name, description, features, labels, metrics):
    from hsml.schema import Schema
    from hsml.model_schema import ModelSchema
    import os
    import joblib
    import shutil

    mr = project.get_model_registry()

    model_dir= name + "_model"
    if not os.path.isdir(model_dir):
        os.mkdir(model_dir)
    pickle= name + '_model.pkl'

    joblib.dump(model, model_dir + "/" + pickle)

    input_example = features.sample()
    input_schema = Schema(features)
    output_schema = Schema(labels)
    model_schema = ModelSchema(input_schema, output_schema)

    diabetes_prediction_model = mr.python.create_model(
        name="diabetes_prediction_model",
        metrics=metrics,
        model_schema=model_schema,
        input_example=input_example,
        description=description)

    # Save all artifacts in the model directory to the model registry
    diabetes_prediction_model.save(model_dir)


register_python_model(model, "diab_pred", "Diabetes Predictor model", X_train, y_train, metrics)


Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/607765 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/35 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/613 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/112801/models/diabetes_prediction_model/1
